# Orbit Visualizer
This notebook will be used to visualize the orbits found ni the reference population dataset for Planet 9/X. It is currently a work in progress.

In [38]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import ipywidgets as ipy

### Creating table with solar system planets

In [39]:
solar_system = [
    { "name": "Mercury", "a": 0.3871, "e": 0.206, "inc": 7.00, "varpi": 29.124, "color": "gray" },
    { "name": "Venus", "a": 0.7233, "e": 0.007, "inc": 3.39, "varpi": 54.884, "color": "gold" },
    { "name": "Earth", "a": 1, "e": 0.017, "inc": 0.00, "varpi": 114.207, "color": "green" },
    { "name": "Mars", "a": 1.5273, "e": 0.093, "inc": 1.85, "varpi": 286.5, "color": "red" },
    { "name": "Jupiter", "a": 5.2028, "e": 0.048, "inc": 1.31, "varpi": 273.867, "color": "orange" },
    { "name": "Saturn", "a": 9.5388, "e": 0.056, "inc": 2.49, "varpi": 339.392, "color": "yellow" },
    { "name": "Uranus", "a": 19.1914, "e": 0.046, "inc": 0.77, "varpi": 96.998, "color": "blue" },
    { "name": "Neptune", "a": 30.0611, "e": 0.010, "inc": 1.77, "varpi": 273.187, "color": "purple" },
]

### Function for plotting the orbits

In [40]:
def visualize_orbits(orbits, count, elevation = 30, azimuth = 40, zoom = 1):

    ax = plt.figure(figsize=(8, 8)).add_subplot(projection='3d')
    max_extent = 0
    
    for i in range(0, count + 8):

        # Determines the shape of the given orbit
        a = orbit_data[i]['a']        # Semi-major axis
        e = orbit_data[i]['e']        # Eccentricity
        b = a * np.sqrt(1 - e ** 2)   # Semi-minor axis
        offset = a * e                # Position of the orbit's focus

        # Creates an ellipse representing the given orbit
        t = np.linspace(0, 2 * np.pi, 100)
        x = offset + a * np.cos(t)
        y = b * np.sin(t)
        
        # Rotates the ellipse both in its inclination and it's argument of perihelion
        theta = orbit_data[i]['varpi'] * np.pi / 180
        phi = orbit_data[i]['inc']  * np.pi / 180
        rot_x = x * np.cos(theta) - y * np.sin(theta)
        y = x * np.sin(theta) + y * np.cos(theta)
        x = rot_x * np.cos(phi)
        z = rot_x * -np.sin(phi)
        
        max_extent = max(max_extent, np.abs(np.min(x)), np.abs(np.min(y)), np.abs(np.min(z)), np.max(x), np.max(y), np.max(z))
        
        if orbit_data[i]['color'] == "random":
            plt.plot(x, y, z, label = orbit_data[i]['name'])
        else:
            plt.plot(x, y, z, label = orbit_data[i]['name'], color=orbit_data[i]['color'])
        
        plt.grid(color = 'lightgray', linestyle = '--')
    
    # Makes the 3d plot area square and allows for zooming in and out
    log_zoom = np.log10(1 + zoom)
    max_log_zoom = np.log10(2)
    zoom = log_zoom / max_log_zoom
    extent = ((1 - zoom) * max_extent) + (zoom * 0.01) 
    ax.set_xlim(-extent, extent)
    ax.set_ylim(-extent, extent)
    ax.set_zlim(-extent, extent)
    
    # Adds the sun in the center, to scale
    u = np.linspace(0, 2 * np.pi, 200)
    v = np.linspace(0, np.pi, 200)
    x = 0.00485 * np.outer(np.cos(u), np.sin(v))
    y = 0.00485 * np.outer(np.sin(u), np.sin(v))
    z = 0.00485 * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, color="orange")

    ax.view_init(elevation, azimuth)

    plt.tight_layout()
    plt.legend(loc="upper right");

### Plots interactively

In [41]:
# Randomize the reference population data
reference_population = pd.read_csv("reference_population.csv", comment = "#")
reference_population = reference_population.sample(frac = 1).reset_index(drop = True)

orbit_data = solar_system.copy()
for i in range(100):
    orbit_data.append({
        "name": "Orbit " + str(reference_population.at[i, 'index']),
        "a": reference_population.at[i, ' a'],
        "e": reference_population.at[i, ' e'],
        "inc": reference_population.at[i, ' inc'],
        "varpi": reference_population.at[i, ' varpi'],
        "color": "random"
    })
    
ipy.interact(
    visualize_orbits, 
    orbits = ipy.fixed(orbit_data), 
    count = ipy.IntSlider(0, min=0, max=10), 
    elevation = ipy.FloatSlider(30, min=0, max=90, step=0.5),
    azimuth = ipy.FloatSlider(60, min=0, max=90, step=0.5), 
    zoom = ipy.FloatSlider(0, min = 0, max=1, step=0.01)
);

interactive(children=(IntSlider(value=0, description='count', max=10), FloatSlider(value=30.0, description='el…

### A failed attempt to plot using plotly

In [5]:
import plotly.io as pio
import plotly.graph_objs as go
pio.renderers.default = 'jupyterlab'

def visualize_orbits(count = 0):
    
    global reference_population

    # Add a random orbit to the solar system data
    reference_population = reference_population.sample(frac = 1).reset_index(drop = True)
    orbit_data = solar_system.copy()

    for i in range(count):
        orbit_data.append({
            "name": "Orbit " + str(reference_population.at[i, 'index']),
            "a": reference_population.at[i, ' a'],
            "e": reference_population.at[i, ' e'],
            "inc": reference_population.at[i, ' inc'],
            "varpi": reference_population.at[i, ' varpi'],
        })
        
    fig = go.Figure()
        
    for i in range(0, len(orbit_data)):

        t = np.linspace(0, 2 * np.pi, 100)
        a = orbit_data[i]['a'] # Semi-major axis
        e = orbit_data[i]['e'] # Eccentricity
        offset = a * e # Center of the orbit
        b = a * np.sqrt(1 - e ** 2) # Semi-minor axis

        theta = orbit_data[i]['varpi'] * np.pi / 180
        phi = orbit_data[i]['inc']  * np.pi / 180

        x = offset + a * np.cos(t)
        y = b * np.sin(t)

        theta = orbit_data[i]['varpi'] * np.pi / 180
        phi = orbit_data[i]['inc']  * np.pi / 180

        rot_x = x * np.cos(theta) - y * np.sin(theta)
        rot_y = x * np.sin(theta) + y * np.cos(theta)

        x = rot_x * np.cos(phi)
        y = rot_y
        z = rot_x * -np.sin(phi)
        
        fig.add_trace(go.Scatter3d(x=x,y=y,z=z, mode='lines'))

    fig.show()
    
visualize_orbits()

ModuleNotFoundError: No module named 'plotly'